In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from collections import Counter
import ast

## Data Imports

In [2]:
misclassified = pd.read_csv("../analysis/val_misclassified_epoch5.csv")
print(misclassified.shape)
misclassified.head()

(114924, 5)


,text,label_id,true_label,pred_label,top3_labels
0,"This team are scared of that lot, its disgust...",23,23,10,"[10, 5, 9]"
1,* Song Credit: \n aka,21,21,17,"[17, 12, 38]"
2,“you already are” \ni love when blitz is a lit...,34,34,7,"[7, 31, 30]"
3,I see and so many friends tonight trying to ...,26,26,6,"[6, 26, 22]"
4,Thanks for including our Super Savings Accoun...,1,1,37,"[37, 0, 39]"


In [3]:
cleaned = pd.read_csv('../data/cleaned.csv')
label_to_emoji = (
    cleaned[["label_id", "emoji_char"]]
    .drop_duplicates()
    .set_index("label_id")["emoji_char"]
    .to_dict()
)

## Misclassification Analysis

In [4]:
errors = misclassified[misclassified["true_label"] != misclassified["pred_label"]]

pair_counts = (
    errors
    .groupby(["true_label", "pred_label"])
    .size()
    .reset_index(name="count")
)

top10 = pair_counts.sort_values("count", ascending=False).head(10)

top10["true_emoji"] = top10["true_label"].apply(lambda x: label_to_emoji.get(x, ""))
top10["pred_emoji"] = top10["pred_label"].apply(lambda x: label_to_emoji.get(x, ""))
top10["pair"] = top10["true_emoji"] + " → " + top10["pred_emoji"]

top10

,true_label,pred_label,count,true_emoji,pred_emoji,pair
879,22,21,713,🥳,🎉,🥳 → 🎉
840,21,22,540,🎉,🥳,🎉 → 🥳
40,1,2,467,✔,✅,✔ → ✅
728,18,27,460,😭,💀,😭 → 💀
79,2,1,398,✅,✔,✅ → ✔
773,19,33,382,🫠,🥲,🫠 → 🥲
783,20,3,382,🖕,🤡,🖕 → 🤡
952,24,16,380,❤,🐣,❤ → 🐣
1071,27,18,380,💀,😭,💀 → 😭
1225,31,16,372,🥰,🐣,🥰 → 🐣


In [5]:
df_not_in_top3 = misclassified[misclassified.apply(lambda r: r["true_label"] not in ast.literal_eval(r["top3_labels"]), axis=1)]

print(df_not_in_top3.shape)

hard_errors = df_not_in_top3[df_not_in_top3["true_label"] != df_not_in_top3["pred_label"]]

pair_counts = (
    hard_errors
    .groupby(["true_label", "pred_label"])
    .size()
    .reset_index(name="count")
)

top5 = pair_counts.sort_values("count", ascending=False).head(5).copy()

top5["true_emoji"] = top5["true_label"].apply(lambda x: label_to_emoji.get(x, ""))
top5["pred_emoji"] = top5["pred_label"].apply(lambda x: label_to_emoji.get(x, ""))

top5["pair"] = top5["true_emoji"] + " → " + top5["pred_emoji"]

top5

(86268, 5)


,true_label,pred_label,count,true_emoji,pred_emoji,pair
416,10,27,197,😂,💀,😂 → 💀
1333,34,7,191,✨,🥹,✨ → 🥹
1001,25,27,179,🤣,💀,🤣 → 💀
767,19,27,175,🫠,💀,🫠 → 💀
943,24,7,170,❤,🥹,❤ → 🥹
